# Resumo do código

### <u>Código que gera os ficheiros para estudo do comportamento da Delta</u>
---
O objectivo é receber dados da Delta e devolver um conjunto de métricas para prever roturas. Devolve um ficheiro que pode entrar no código 1 para juntar aos dados dos ninjas.

---
- Inputs

> __Dados completos da Delta em pastas de ficheiros__ (de azul a verde)
> - Stocks e trânsito, Sellout do dia anterior

                    ou

> __Ficheiro já completo__ (de vermelho a verde)
> - Stocks e trânsito, Sellout do dia anterior

- Outputs

> __Ficheiro com produtos em causa__ em formato Long

> __Métricas novas:__
> - Roturas de Stock e Pré-rotura
> - Sinal
> - Ciclos e Adequação de Stock
> - MSA (média de sellouts 10 dias antes)
> - STK (Stock disponível + trânsito)
> - (Novo) Balanço médio, mediano, liberal e conservador 
> - (Novo) Dias para a rotura de stock e de prateleira


In [3]:
%%time
import pandas as pd
import os
import glob
import numpy as np
import datetime
from IPython.display import Audio

def escrever_csv(dfa, nome):
    dfa.to_csv(f'C:\\Users\\Chip7\\Desktop\\B&N\\Dados\\2.Delta\\Sonae\\Stocks\\{nome}.csv', index=False)

CPU times: total: 46.9 ms
Wall time: 238 ms


---

# <font color=blue><u>Ler os ficheiros na pasta</u><font>

- Demora 30 min

In [5]:
%%time

a=0
# Definir a directória dos ficheiros

directoria = 'C:\\Users\\Chip7\\Desktop\\B&N\\Dados\\2.Delta\\Sonae\\Stocks\\Stocks2023\\DadosJanMaio'  #xlsx


# Definir listas vazias para meter os diferentes dataframes
ficheiros = [file for file in os.listdir(directoria)]
dataframes = []



# Iterar para todos os ficheiros excel

#xlsx
for file in ficheiros:
    a+=1
    print (a)
    
    # Read the data from the Excel file into a DataFrame
    df = pd.read_excel(directoria + file)

    # Add the DataFrame to the list
    dataframes.append(df)

        
# Concatenate todos os dfs num único DataFrame
original1 = pd.concat(dataframes, ignore_index=True)
#original1['DATA']=pd.to_datetime(original1['DATA'], format='%d-%m-%Y')
original1['DATA'] = pd.to_datetime(original1['DATA'], unit='D', origin=datetime.datetime(1899, 12, 30))


freq = 440  # Hz
duration = 1  # seconds
samples = int(duration * 44100)  # 44100 samples per second
t = np.linspace(0, duration, samples, False)
note = np.sin(freq * t * 2 * np.pi)
Audio(note, rate=44100, autoplay=True)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
CPU times: total: 2min 29s
Wall time: 4min 26s


In [7]:
%%time

#8 min



directory2 = 'C:\\Users\\Chip7\\Desktop\\B&N\\Dados\\2.Delta\\Sonae\\Stocks\\Stocks2023\\DadosJunhOut' #xlsb
a=0
#xlsb
dataframes2 = []
for filename in os.listdir(directory2):
    if filename.endswith(".xlsb") and filename.startswith("Info_DELTA_"):
        a+=1
        print (a)
        # Read the data from the Excel file into a DataFrame
        filepath = os.path.join(directory2, filename)
        df = pd.read_excel(filepath, engine='pyxlsb')
        
        # Add the DataFrame to the list
        dataframes2.append(df)   

original2 = pd.concat(dataframes2, ignore_index=True)
original2['DATA'] = pd.to_datetime(original2['DATA'], unit='D', origin=datetime.datetime(1899, 12, 30))

freq = 440  # Hz
duration = 1  # seconds
samples = int(duration * 44100)  # 44100 samples per second
t = np.linspace(0, duration, samples, False)
note = np.sin(freq * t * 2 * np.pi)
Audio(note, rate=44100, autoplay=True)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
CPU times: total: 2min 50s
Wall time: 4min 20s


In [10]:
#original1['DATA']=pd.to_datetime(original1['DATA'], format='%d-%m-%Y')
original = pd.concat([original1, original2], ignore_index=True)

In [11]:
original

,DATA,FORN/AGENTE/ENT,DESC_FORN/AGENTE/ENT,FORN,DESC_FORN,CAT,ARTIGO,DESC_ARTIGO,STATUS_ARTIGO,PACK,...,STORE_NAME,FLUXO,SPK,SOH,INTRANSIT,EXPECTED,Enc PBL + DIRECTO,PRES_STOCK,VND (D-1),EAN
0,2023-01-01,14304,ENT. MAIA AMB.,1748,MANUEL RUI AZINHAIS,1403,2004739,CAFÉ DELTA EXPRESSO BAR GRAO 1KG,A,NaN,...,CNT MATOSINHOS,PBL,1,61.0,0,0,0,16,0.0,5601082006345
1,2023-01-01,14299,MCH ENT.AZAMBUJA AMB.,1748,MANUEL RUI AZINHAIS,1403,2004739,CAFÉ DELTA EXPRESSO BAR GRAO 1KG,A,NaN,...,CNT AMADORA,PBL,1,98.0,0,0,0,16,1.0,5601082006345
2,2023-01-01,14299,MCH ENT.AZAMBUJA AMB.,1748,MANUEL RUI AZINHAIS,1403,2004739,CAFÉ DELTA EXPRESSO BAR GRAO 1KG,A,NaN,...,CNT CASCAIS,PBL,1,34.0,0,0,0,16,2.0,5601082006345
3,2023-01-01,14304,ENT. MAIA AMB.,1748,MANUEL RUI AZINHAIS,1403,2004739,CAFÉ DELTA EXPRESSO BAR GRAO 1KG,A,NaN,...,CNT GAIASHOPPING,PBL,1,70.0,0,0,0,16,4.0,5601082006345
4,2023-01-01,14299,MCH ENT.AZAMBUJA AMB.,1748,MANUEL RUI AZINHAIS,1403,2004739,CAFÉ DELTA EXPRESSO BAR GRAO 1KG,A,NaN,...,CNT LEIRIA,PBL,1,60.0,0,0,0,16,2.0,5601082006345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7203127,2023-10-03,14304,ENT. MAIA AMB.,1748,MANUEL RUI AZINHAIS,1403,7844707,CHOCOLATE QUENTE CREMOSO CARAMELO,A,NaN,...,CNT ARRABIDA,PBL,1,0.0,0,0,0,12,0.0,2100128533614
7203128,2023-10-03,14299,MCH ENT.AZAMBUJA AMB.,1748,MANUEL RUI AZINHAIS,1403,7844707,CHOCOLATE QUENTE CREMOSO CARAMELO,A,NaN,...,CNT BARREIRO,PBL,1,0.0,0,0,0,12,0.0,2100128533614
7203129,2023-10-03,1748,MANUEL RUI AZINHAIS,1748,MANUEL RUI AZINHAIS,1701,5384891,VB ADEGA MAYOR RES REG ALENT 75CL,A,NaN,...,CBD CAMPO MAIOR,DIRECTO,1,11.0,0,0,0,6,0.0,5607623012905
7203130,2023-10-03,1748,MANUEL RUI AZINHAIS,1748,MANUEL RUI AZINHAIS,1701,6952867,VT DIZERES ADEGA MAYOR ALENTEJO 75CL,A,NaN,...,CBD CAMPO MAIOR,DIRECTO,1,71.0,0,0,0,24,0.0,5607623011106


# Limitar e Corrigir o ano

In [13]:
%%time
dfTotal = original.loc[:, ['DATA', 'EAN', 'DESC_ARTIGO', 'STORE', "STORE_NAME", 
                         "SOH", "INTRANSIT", "EXPECTED", "PRES_STOCK", "VND (D-1)", "FLUXO"]].copy()

dfTotal = dfTotal.rename(columns = {"VND (D-1)": "SELLOUT_1_Dias_Antes", 
                                    "SOH":"STOCK_1_Dias_Antes",
                                    "INTRANSIT": "INTRANSIT_1_Dias_Antes", 
                                    "EXPECTED": "EXPECTED_1_Dias_Antes", 
                                    "PRES_STOCK": "PRES_STOCK_1_Dias_Antes"})

CPU times: total: 609 ms
Wall time: 751 ms


# Organizar dados
> Testar a qualidade da leitura

In [16]:
dfTotal = dfTotal.sort_values(by = ["DATA", "STORE", "DESC_ARTIGO"])
dfTotal[dfTotal.DESC_ARTIGO!="100% SALICÓRNIA MOIDA 45G"][["DATA","DESC_ARTIGO","STORE"]].head(50)#.iloc[150:200]           #combined_df.loc(combined_df['DESC_ARTIGO'] == "100% SALICÓRNIA MOIDA 45G").head(20)

,DATA,DESC_ARTIGO,STORE
1826993,2023-03-11,AZEITONA BRITADA QAMPO 5KG CC,8
1854471,2023-03-12,AZEITONA BRITADA QAMPO 5KG CC,8
1881937,2023-03-13,AZEITONA BRITADA QAMPO 5KG CC,8
1908883,2023-03-14,AZEITONA BRITADA QAMPO 5KG CC,8
1937303,2023-03-15,AZEITONA BRITADA QAMPO 5KG CC,8
1965732,2023-03-16,AZEITONA BRITADA QAMPO 5KG CC,8
1994133,2023-03-17,AZEITONA BRITADA QAMPO 5KG CC,8
2022537,2023-03-18,AZEITONA BRITADA QAMPO 5KG CC,8
2050780,2023-03-19,AZEITONA BRITADA QAMPO 5KG CC,8
2078878,2023-03-20,AZEITONA BRITADA QAMPO 5KG CC,8


> Colunas para sellouts e stocks do dia

In [18]:
dfTotal["SELLOUT_1_Dias_Antes"] = np.where(dfTotal["SELLOUT_1_Dias_Antes"]<0, 0, dfTotal["SELLOUT_1_Dias_Antes"])
# Sellout do dia
dfTotal['SELLOUT'] = dfTotal.groupby(["STORE","EAN"])['SELLOUT_1_Dias_Antes'].shift(-1)

# Stock do dia
dfTotal['STOCK'] = dfTotal.groupby(["STORE","EAN"])['STOCK_1_Dias_Antes'].shift(-1)

dfTotal['INTRANSIT'] = dfTotal.groupby(["STORE","EAN"])['INTRANSIT_1_Dias_Antes'].shift(-1)
dfTotal['EXPECTED'] = dfTotal.groupby(["STORE","EAN"])['EXPECTED_1_Dias_Antes'].shift(-1)
dfTotal['PRES_STOCK'] = dfTotal.groupby(["STORE","EAN"])['PRES_STOCK_1_Dias_Antes'].shift(-1)

> Ordenar as colunas

In [19]:
df_columns = dfTotal.columns.tolist()
[f"{index}: {column}" for index, column in enumerate(df_columns)]

['0: DATA',
 '1: EAN',
 '2: DESC_ARTIGO',
 '3: STORE',
 '4: STORE_NAME',
 '5: STOCK_1_Dias_Antes',
 '6: INTRANSIT_1_Dias_Antes',
 '7: EXPECTED_1_Dias_Antes',
 '8: PRES_STOCK_1_Dias_Antes',
 '9: SELLOUT_1_Dias_Antes',
 '10: FLUXO',
 '11: SELLOUT',
 '12: STOCK',
 '13: INTRANSIT',
 '14: EXPECTED',
 '15: PRES_STOCK']

In [20]:
dfTotalOrg = dfTotal.iloc[:, np.r_[:5, 12, 5, 11,9, 13, 6, 14,7, 15,8, 10]]

In [21]:
[f"{index}: {column}" for index, column in enumerate(dfTotalOrg.columns)]

['0: DATA',
 '1: EAN',
 '2: DESC_ARTIGO',
 '3: STORE',
 '4: STORE_NAME',
 '5: STOCK',
 '6: STOCK_1_Dias_Antes',
 '7: SELLOUT',
 '8: SELLOUT_1_Dias_Antes',
 '9: INTRANSIT',
 '10: INTRANSIT_1_Dias_Antes',
 '11: EXPECTED',
 '12: EXPECTED_1_Dias_Antes',
 '13: PRES_STOCK',
 '14: PRES_STOCK_1_Dias_Antes',
 '15: FLUXO']

# Escrever

In [22]:
%%time
escrever_csv(dfTotalOrg, "Stocks_Delta_2023_Limpo")

CPU times: total: 18.1 s
Wall time: 22.7 s


# <font color=green>Ficheiros Lidos<font>

---

# Colunas de métricas interessantes

> - ROTURA

In [27]:
# Definir coluna de rotura (se stock menor ou igual a 0 e existe Linear)

dfFinal["ROTURA"] = np.where((dfFinal["STOCK"] <= 0) & (dfFinal["PRES_STOCK"] > 0), 1, 0)

> - PRÉ_ROTURA

In [28]:
# Definir coluna de rotura (se stock menor ou igual a 0)

dfFinal["PRE_ROTURA"] = (dfFinal["STOCK"] < dfFinal["PRES_STOCK"]).astype(int)

# Métricas até 10 dias antes:

- INSTRANSIT
- EXPECTED
- SELLOUT
- CICLOS
- Dias para Rotura
- Adequação

In [29]:
# Quantos dias antes:

diaI=1         #dia inicial
diaF=10        #dia final

> Função

In [30]:
# Função para colunas de dias anteriores
def dias(df, dia, coluna):         #dia é quantos dias antes
    
    #
    a=int(dia)

    valores = df.groupby(['DESC_ARTIGO', 'STORE'])[coluna].transform(lambda x: x.shift(a))
    valores[:a] = np.nan
    
    df.loc[:,'%s_%s_Dias_Antes' % (coluna, a)] = valores

> - SELLOUTS

In [31]:
%%time
# Usar função para sellouts até 10 dias antes

for i in range(diaI+1, diaF+1):
    dias(dfFinal, i, "SELLOUT")

CPU times: total: 6.39 s
Wall time: 6.84 s


> - STOCKS

In [32]:
# Usar função para Stocks até 10 dias antes

for i in range(diaI+1, diaF+1):
    dias(dfFinal, i, "STOCK")

> - INTRANSIT e EXPECTED

In [33]:
# Usar função para Trânsito até 10 dias antes

for i in range(diaI, diaF+1):
    dias(dfFinal, i, "INTRANSIT")
    
for i in range(diaI, diaF+1):
    dias(dfFinal, i, "EXPECTED")

> - STK

In [34]:
# STK do dia = soma dos stocks em loja com os stocks em trânsito no próprio dia

dfFinal["STK"] = dfFinal["STOCK"] + dfFinal["INTRANSIT"] + dfFinal["EXPECTED"]

for i in range(diaI, diaF+1):
    dias(dfFinal, i, "STK")

> - MSA

In [35]:
# MSA do dia = média dos sellouts dos 10 dias anteriores ao dia em causa

dfFinal["MSA10"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).mean())
dfFinal["MSA10Dp"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).std())

for i in range(diaI, diaF+1):
    dias(dfFinal, i, "MSA10")

    
dfFinal["MSA20"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=20).mean())
dfFinal["MSA20Dp"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=20).std())
  
for i in range(diaI, diaF+1):
    dias(dfFinal, i, "MSA20")

> - CICLOS

In [36]:
# Coluna de Ciclos de reposição

dfFinal["CICLOS"] = dfFinal["STOCK"]/dfFinal["PRES_STOCK"]

for i in range(diaI, diaF+1):
    dias(dfFinal, i, "CICLOS")

# Final2
> - Dias para rotura de Stock

In [37]:
dfFinal2 = dfFinal.copy()
# Dias para a rotura mas com o Sellout médio (móvel) dos últimos 10 dias 
dfFinal2["Dias_para_Rotura_Stock"] = dfFinal2["STOCK"] / dfFinal2["MSA10"]

for i in range(diaI, diaF+1):
    dias(dfFinal2, i, "Dias_para_Rotura_Stock")

> - Dias para rotura de Linear

In [38]:
# Definir a métrica: Preslinear / med(Sellouts 10 dias)
dfFinal2['Dias_Duração_Linear'] = dfFinal2["PRES_STOCK"] / dfFinal2["MSA10"]

for i in range(diaI, diaF+1):
    dias(dfFinal2, i, "Dias_Duração_Linear")

> - Adequação de Stock

In [39]:
# Coluna de adequação de stock


dfFinal2["Adequação"]= np.where(dfFinal2["CICLOS"] > 1.1, "Stock Suficiente", 
                      np.where((dfFinal2["CICLOS"] <= 1.1) & (dfFinal2["INTRANSIT"]+dfFinal2["EXPECTED"]+dfFinal2["STOCK"]>=dfFinal2["PRES_STOCK"]), "Stock Insuf c Forn Adequado", 
                      np.where((dfFinal2["CICLOS"] <= 1.1) & (dfFinal2["INTRANSIT"]+dfFinal2["EXPECTED"]+dfFinal2["STOCK"]<dfFinal2["PRES_STOCK"]), "Stock Insuf c Forn Desadequado", 
                      "")))

for i in range(diaI, diaF+1):
    dias(dfFinal2, i, "Adequação")

>- Balance: sellout / soma stock disponível mais transito.

In [40]:
# Colunas de balanço


# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal2["Balance"] =  dfFinal2["MSA10"] / dfFinal2["STK"]

for i in range(diaI, diaF+1):
    
    
    valores = (i+1) * dfFinal2["Balance"].shift(i)
    valores[:i] = np.nan
    
    dfFinal2.loc[:,'%s_%s_Dias_Antes' % ("Balance", i)] = valores
    
    


In [42]:
a=0
b=1

for i in range (21):
    if i == 0:
        a=1
        print(a)
    else:
        print(b)
        c = a + b
        a = b
        b = c


1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765
10946


>- Balance optimizado

In [41]:


    
dfFinal2["Balance_Optimized"] = np.where((dfFinal2["MSA10Dp"] / dfFinal2["MSA10"]) * 100 > 100, dfFinal2["MSA20"] / dfFinal2["STK"],
                                dfFinal2["MSA10"] / dfFinal2["STK"])   

for i in range(diaI, diaF+1):
    
    
    valores = (i+1) * dfFinal2["Balance_Optimized"].shift(i)
    valores[:i] = np.nan
    
    dfFinal2.loc[:,'%s_%s_Dias_Antes' % ("Balance_Optimized", i)] = valores

> - Mediano é com o mínimo dos ultimos 10 dias

In [42]:
# Coluna de adequação de stock


# MSA_med do dia = mediana dos sellouts dos 10 dias anteriores (exclui o próprio dia)
# dfFinal2["MdSA"] = dfFinal2.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).median())


# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal2["Balance_Mediano"] =  dfFinal2.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).median()) / dfFinal2["STK"]

for i in range(diaI, diaF+1):
    
    
    valores = (i+1) * dfFinal2["Balance_Mediano"].shift(i)
    valores[:i] = np.nan
    
    dfFinal2.loc[:,'%s_%s_Dias_Antes' % ("Balance_Mediano", i)] = valores



> - Liberal é com o mínimo dos ultimos 10 dias

In [43]:
# Liberal


# MSA do dia = média dos stocks dos 10 dias anteriores (exclui o próprio dia)
#dfFinal2["LSA"] = dfFinal2.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).min())


# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal2["Balance_Liberal"] =  dfFinal2.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).min()) / dfFinal2["STK"]

for i in range(diaI, diaF+1):
    
    
    valores = (i+1) * dfFinal2["Balance_Liberal"].shift(i)
    valores[:i] = np.nan
    
    dfFinal2.loc[:,'%s_%s_Dias_Antes' % ("Balance_Liberal", i)] = valores



> - Conservador é com o máximo dos ultimos 10 dias

In [44]:
# Conservador

# MSA_max do dia = máximo dos stocks dos 10 dias anteriores (exclui o próprio dia)
#dfFinal2["CSA"] = dfFinal2.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).max())
 
    
# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal2["Balance_Conservador"] =  dfFinal2.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).max()) / dfFinal2["STK"]

for i in range(diaI, diaF+1):
    
    # Multiplicar os balances pelo número de dias+1 antes do dia actual
    valores = (i+1) * dfFinal2["Balance_Conservador"].shift(i)
    valores[:i] = np.nan
    
    dfFinal2.loc[:,'%s_%s_Dias_Antes' % ("Balance_Conservador", i)] = valores



In [28]:
dfFinal2[["DATA","STORE","DESC_ARTIGO","STOCK","INTRANSIT","EXPECTED","SELLOUT","Balance","Balance_4_Dias_Antes","Balance_Liberal","Balance_Liberal_4_Dias_Antes","Balance_Mediano_4_Dias_Antes","Balance_Conservador_4_Dias_Antes"]].tail(20)

,DATA,STORE,DESC_ARTIGO,STOCK,INTRANSIT,EXPECTED,SELLOUT,Balance,Balance_4_Dias_Antes,Balance_Liberal,Balance_Liberal_4_Dias_Antes,Balance_Mediano_4_Dias_Antes,Balance_Conservador_4_Dias_Antes
11988342,2023-04-21,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,38.0,0,0,5.0,0.128947,0.568627,0.026316,0.098039,0.490196,1.078431
11988343,2023-04-22,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,33.0,0,0,9.0,0.151515,0.540816,0.030303,0.102041,0.459184,1.122449
11988344,2023-04-23,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,24.0,0,0,5.0,0.233333,0.600000,0.041667,0.111111,0.500000,1.222222
11988345,2023-04-24,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,19.0,0,6,2.0,0.224000,0.756410,0.040000,0.256410,0.641026,1.410256
11988346,2023-04-25,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,17.0,6,6,1.0,0.182759,0.644737,0.034483,0.131579,0.592105,1.315789
11988347,2023-04-26,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,22.0,6,0,0.0,0.157143,0.757576,0.035714,0.151515,0.757576,1.515152
11988348,2023-04-27,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,22.0,6,6,0.0,0.102941,1.166667,0.000000,0.208333,1.041667,2.083333
11988349,2023-04-28,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,28.0,6,18,2.0,0.063462,1.120000,0.000000,0.200000,1.000000,2.000000
11988350,2023-04-29,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,32.0,0,18,5.0,0.062000,0.913793,0.000000,0.172414,0.862069,1.724138
11988351,2023-04-30,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,27.0,0,24,14.0,0.058824,0.785714,0.000000,0.178571,0.803571,1.607143


# Escrever

- 0's e 1's

In [45]:
# Novo dataframe que apenas inclui dias em que existe a 1ª rotura e o dia anterior a essa rotura

# Tirar 1's depois do primeiro
dfFinalLimitado = dfFinal2[~((dfFinal2["ROTURA"] == 1) & (dfFinal2["ROTURA"].shift(1) == 1))].copy()

# Apenas incluir primeira rotura
dfFinalLimitadoRoturas = dfFinal2[((dfFinal2["ROTURA"] == 1) & (dfFinal2["ROTURA"].shift(1) == 0))].copy() #| ((dfFinal["ROTURA"] == 0) & (dfFinal["ROTURA"].shift(-1) == 1))]

- Dias certos

dfFinal2 = dfFinal2.loc[(dfFinal2['DATA'] >= '2023-01-01') ].copy()
df_DiasCertos = dfFinalLimitado.loc[(dfFinalLimitado['DATA'] >= '2023-01-01') ].copy()
df_RoturasDiasCertos = dfFinalLimitadoRoturas.loc[(dfFinalLimitadoRoturas['DATA'] >= '2023-01-01') ].copy()

- Passar para csv